# Usage of `src.data.opensky_query` module.

This notebook outlies the basic usage of the `src.data.opensky_query` module. Used to query flight data and and state vector data from the opensky database.

**Definitions**
 - Flight Data: Information about the flight, icao24 number, callsign, first and last seen times.
 - State Vector Data: Information about flight's position and movement at given points of time.
 
**Requirements**
 - A OpenSky Login and access to the OpenSky Database. Credentials can be aquired by contacting the administrators of the [website](https://opensky-network.org/about/contact).
 - A `.yml` file containing the credentials to access the OpenSky Database
 - A `.yml` file being a copy of `config/config_template.yml` contaning information about the flights to be downloaded, as well as a path to the credentials config file

**Helpful Links**

 - [Date-Time Unix-Time Converter](https://www.unixtimestamp.com/)

 - [A Quick Guide To OpenSky's Impala Shell](https://opensky-network.org/data/impala)


## Setting up access to database

This section sets up the basic access to the database by loading credentials and configuration files into a custom Client object.

In [5]:
import sys
# This variable should indicate the path from this Jupyter Notebook to the root directory of the repo.
root_path = '../'
# Adds the repo's root to the list of paths
sys.path.append(root_path)

# Package to define and interpret dates
import datetime
# Package to read yml files
import yaml
# Package to handle file paths
import os
# Package for downloading opensky data 
from src.data import opensky_query
# Utilities package
from src.common import utils

# Normalizing all paths to work on all operational systems
root_path = os.path.normpath(root_path) # Path from this notebook to the root directory
config_path_from_root = os.path.normpath('config/config_tutorial.yml') # Path from root to the desired config file
config_path = os.path.join(root_path, config_path_from_root) # Defining path from this notebook to config file

# Loading config file
with open(config_path, 'r',  encoding='utf8') as file:
    config = yaml.safe_load(file)
        
# Defining credentials filepath
credentials_file_from_root = os.path.normpath(config['base-configs']['opensky-credentials'])
credentials_file = os.path.join(root_path, credentials_file_from_root) # Defining path from this notebook to credentials file

# Loading credentials file
with open(credentials_file, 'r',  encoding='utf8') as file:
    credentials = yaml.safe_load(file)

# Creates an instance of a logger class to log all that happens, optional (but encouraged).
logger = utils.Logger(config)

# Creates an instace of the Querier class used for querying the opensky database
opensky_querier = opensky_query.Querier(
    credentials,
    config,
    logger = logger)

## Loading the Flight Data for flights in the airports and dates specified

The code below downloads the flight data for all found flights between the dates specified and between the airports specified. The result is given as a Pandas DataFrame

The columns of the dataframe are:
 - time: Recorded Unix UTC time of when the data was gathered
 - lat: Latitude of the aircraft at the given time
 - lon: Longitude of the aircraft at the given time
 - baroaltitude: Altitude of the aircraft according to the internal barometer at the given time
 - geoaltitude: Altitude of the aircraft according to the internal gps at the given time


In [6]:
# Looking at all flights from Baton Rouge to Dallas from `Jan 1, 2022` to `Jan 2, 2022`
departure_airport = 'KBTR'
arrival_airport = 'KDFW'
start_date = datetime.date(2022, 1, 1)
end_date = datetime.date(2022, 1, 2)

# This code will query the database and return the flights requested.
# As mentioned in the wiki, sometimes there are missing files for some days, which the code handles and adds as an exception before repeating the query
flights = opensky_querier.query_flight_data(
          {'departure_airport': departure_airport,
          'arrival_airport': arrival_airport}, 
          {'start': start_date, 
          'end': end_date})

flights

2023/11/05 18:40:14 : Querying data for flights from KBTR to KDFW between the dates 2021-12-31 18:00:00 and 2022-01-01 18:00:00
2023/11/05 18:40:36 : Querying: SELECT firstseen, lastseen, callsign, icao24, estdepartureairport, estarrivalairport, day
    FROM flights_data4 
    WHERE estdepartureairport = 'KBTR' 
    AND estarrivalairport = 'KDFW'
    AND day >= 1640995200
    AND day <= 1641081600
    ORDER BY firstseen;
2023/11/05 18:41:00 : Querying: SELECT firstseen, lastseen, callsign, icao24, estdepartureairport, estarrivalairport, day
    FROM flights_data4 
    WHERE estdepartureairport = 'KBTR' 
    AND estarrivalairport = 'KDFW'
    AND day >= 1640995200
    AND day <= 1641081600
    ORDER BY firstseen;


,firstseen,lastseen,callsign,icao24,estdepartureairport,estarrivalairport,day
0,1641047161,1641051233,SKW3125,a99b5f,KBTR,KDFW,1640995200
1,1641061714,1641065878,ENY3536,a8dbbd,KBTR,KDFW,1640995200
2,1641089450,1641093689,SKW3185,a972be,KBTR,KDFW,1641081600
3,1641125850,1641130088,SKW3125,a992cf,KBTR,KDFW,1641081600
4,1641147684,1641151763,ENY3536,a81db0,KBTR,KDFW,1641081600
5,1641160980,1641165650,N527RY,a6a47d,KBTR,KDFW,1641081600


## Loading State Vector data for a single flight

The code below donloads a specific flight's state vectors, and returns it in a pandas DataFrame.

The columns of the dataframe are:
 - time: Recorded Unix UTC time of when the data was gathered
 - lat: Latitude of the aircraft at the given time
 - lon: Longitude of the aircraft at the given time
 - baroaltitude: Altitude of the aircraft according to the internal barometer at the given time
 - geoaltitude: Altitude of the aircraft according to the internal gps at the given time


In [7]:
# Let's choose a random flight out of the dataframe we got
flight = flights.sample(1).iloc[0]

# Similarly to querying flight data, there are 'bad hours' which can happen
state_vectors = opensky_querier.query_state_vectors(
                flight['icao24'],
                flight['firstseen'],
                flight['lastseen'])

state_vectors

2023/11/05 18:41:01 : Querying data for statevectors for ICAO24 a8dbbd between the times 2022-01-01 12:28:34 and 2022-01-01 13:37:58
2023/11/05 18:41:23 : Querying: SELECT time, lat, lon, velocity, heading, baroaltitude, geoaltitude, onground, hour
    FROM state_vectors_data4
    WHERE icao24 = 'a8dbbd' 
    AND (time >= 1641061714 AND time <= 1641065878)
    AND (hour > 1641058114 AND hour < 1641069478)
    ORDER BY time;


,time,lat,lon,velocity,heading,baroaltitude,geoaltitude,onground,hour
0,1641061714,NULL,NULL,NULL,NULL,NULL,NULL,false,1641060000
1,1641061715,30.528488,-91.153241,70.935467,224.706178,99.06,NULL,false,1641060000
2,1641061716,30.528488,-91.153241,70.935467,224.706178,99.06,NULL,false,1641060000
3,1641061717,30.528488,-91.153241,70.935467,224.706178,99.06,NULL,false,1641060000
4,1641061718,30.528488,-91.153241,70.935467,224.706178,99.06,NULL,false,1641060000
...,...,...,...,...,...,...,...,...,...
4160,1641065874,32.8548,-97.00995,72.029509,359.181545,388.62,281.94,false,1641063600
4161,1641065875,32.8548,-97.00995,71.515118,359.175657,388.62,281.94,false,1641063600
4162,1641065876,32.8548,-97.00995,71.515118,359.175657,388.62,281.94,false,1641063600
4163,1641065877,32.860153,-97.009991,70.486338,359.163625,358.14,251.46,false,1641063600
